In [1]:
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
from collections import Counter
import os
import glob

import numpy as np

In [ ]:
pathToPos = 'PositiveWithDESSky'
pathToNeg = 'DES/DES_Processed'

In [ ]:
folders = {}
for root, dirs, files in os.walk(pathToPos):
    for folder in dirs:
        key = folder
        value = os.path.join(root, folder)
        folders[key] = value
        
# subf = []
# for folder in folders:
#     subf.append(folder[len(pathToPos)])

In [ ]:
#number of Positive DataPoints
nDT = len(folders)

DataPos = np.zeros([nDT, 3, 100, 100])

# key is name of folder number
# value is the number of the folder to be added to the file name

counter = 0
for key, value in folders.items():

    g_name = get_pkg_data_filename(value + '/' + str(key) + '_g_norm.fits')
    r_name = get_pkg_data_filename(value + '/' + str(key) + '_r_norm.fits')
    i_name = get_pkg_data_filename(value + '/' + str(key) + '_i_norm.fits')

    # g_name = get_pkg_data_filename(value + '/' + str(key) + '_posSky_g.fits')
    # r_name = get_pkg_data_filename(value + '/' + str(key) + '_posSky_r.fits')
    # i_name = get_pkg_data_filename(value + '/' + str(key) + '_posSky_i.fits')
    
    g = fits.open(g_name)[0].data[0:100,0:100]
    r = fits.open(r_name)[0].data[0:100,0:100]
    i = fits.open(i_name)[0].data[0:100,0:100]
    
    DataPos[counter] = [g, r, i] 
    counter += 1
#    if counter > 1500:
#        break

In [ ]:
#Loading negative examples

# r=root, d=directories, f = files

foldersNeg = []
for root, dirs, files in os.walk(pathToNeg):
    for folder in dirs:
        foldersNeg.append(os.path.join(root, folder))

In [ ]:
nDT = len(foldersNeg)
DataNeg = np.zeros([nDT,3,100,100])

for var in range(len(foldersNeg)):

    # g_name = get_pkg_data_filename(foldersNeg[var]+'/g_WCSClipped.fits')
    # r_name = get_pkg_data_filename(foldersNeg[var]+'/r_WCSClipped.fits')
    # i_name = get_pkg_data_filename(foldersNeg[var]+'/i_WCSClipped.fits')    

    g_name = get_pkg_data_filename(foldersNeg[var]+'/g_norm.fits')
    r_name = get_pkg_data_filename(foldersNeg[var]+'/r_norm.fits')
    i_name = get_pkg_data_filename(foldersNeg[var]+'/i_norm.fits')    

    g = fits.open(g_name)[0].data[0:100,0:100]
    r = fits.open(r_name)[0].data[0:100,0:100]
    i = fits.open(i_name)[0].data[0:100,0:100]    
    
    DataNeg[var] = [g, r, i]
#    if var > 1500:
#        break

In [ ]:
def norm(x):
    m = x.mean()
    v = x.std()
    return (x-m)/v

In [ ]:
DataNeg.shape

In [ ]:
DataPos.shape

In [ ]:
from matplotlib import pyplot
im2disp = DataNeg[2].transpose((1,2,0))
pyplot.imshow(im2disp)
pyplot.show()

In [ ]:
im2disp = DataPos[2].transpose((1,2,0))
pyplot.imshow(im2disp)
pyplot.show()

In [ ]:
DataPos.std()

In [ ]:
DataNeg.std()

In [ ]:
AllData = np.vstack((DataPos, DataNeg))

In [ ]:
AllData.std()

In [ ]:
# Gaussian normalization of the data
for i in range(DataPos.shape[0]):
    for j in range(DataPos.shape[1]):
        DataPos[i,j] = norm(DataPos[i,j])
        #print(DataPos.std())

for i in range(DataNeg.shape[0]):
    for j in range(DataNeg.shape[1]):
        DataNeg[i,j] = norm(DataNeg[i,j])
        #print(DataNeg.std())
        

In [ ]:
from matplotlib import pyplot
im2disp = DataNeg[2].transpose((1,2,0))
pyplot.imshow(im2disp)
pyplot.show()

In [ ]:
im2disp = DataPos[2].transpose((1,2,0))
pyplot.imshow(im2disp)
pyplot.show()

In [ ]:
# We need to create train and test "datasets",
# let's say 80% images for training and 20% for test from every group

In [ ]:
DataPos.shape[0]
DataNeg.shape[0]

In [ ]:
import numpy.random as rnd
rnd.seed(2019) #fix seed for reproducibility of results 

listPos = list(np.arange(DataPos.shape[0]))
listPosTest = list(rnd.choice(listPos,int(DataPos.shape[0]*0.2), replace=False))
listPosRem = list(set(listPos)-set(listPosTest))
listPosVal = list(rnd.choice(listPosRem,int(DataPos.shape[0]*0.2), replace=False))
listPosTrain = list(set(listPosRem)-set(listPosVal))


listNeg = list(np.arange(DataPos.shape[0],DataPos.shape[0]+DataNeg.shape[0]))
listNegTest  = list(rnd.choice(listNeg,int(DataNeg.shape[0]*0.2), replace=False))
listNegRem = list(set(listNeg)-set(listNegTest))
listNegVal  = list(rnd.choice(listNegRem,int(DataNeg.shape[0]*0.2), replace=False))
listNegTrain = list(set(listNegRem)-set(listNegVal))

listTest  = listPosTest  + listNegTest
rnd.shuffle(listTest)
listVal  = listPosVal  + listNegVal
rnd.shuffle(listVal)
listTrain = listPosTrain + listNegTrain
rnd.shuffle(listTrain)

In [ ]:
#Now we are ready to create X_train, Y_train and X_test and Y_test

Ntest  = len(listTest)
Nval   = len(listVal)
Ntrain = len(listTrain)

X_train = np.zeros([Ntrain,3,100,100])
Y_train = np.zeros(Ntrain, dtype=int)

X_test = np.zeros([Ntest,3,100,100])
Y_test = np.zeros(Ntest, dtype=int)

X_val = np.zeros([Nval,3,100,100])
Y_val = np.zeros(Nval, dtype=int)


for i in range(Ntest):
    if listTest[i]<DataPos.shape[0]:
        X_test[i] = DataPos[listTest[i]]
        Y_test[i] = 1
    else:
        X_test[i] = DataNeg[listTest[i]-DataPos.shape[0]]
        Y_test[i] = 0

for i in range(Nval):
    if listVal[i]<DataPos.shape[0]:
        X_val[i] = DataPos[listVal[i]]
        Y_val[i] = 1
    else:
        X_val[i] = DataNeg[listVal[i]-DataPos.shape[0]]
        Y_val[i] = 0        
        
for i in range(Ntrain):
    if listTrain[i]<DataPos.shape[0]:
        X_train[i] = DataPos[listTrain[i]]
        Y_train[i] = 1
    else:
        X_train[i] = DataNeg[listTrain[i]-DataPos.shape[0]]
        Y_train[i] = 0

In [ ]:
X_test  = X_test.transpose(0,2,3,1)
X_val   = X_val.transpose(0,2,3,1)
X_train = X_train.transpose(0,2,3,1)

In [ ]:
#Now we are almost ready to create CNN :)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Flatten, Dense, Dropout

In [ ]:
model = Sequential()
model.add(Conv2D(8, kernel_size = (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, epochs=30, batch_size=200, validation_data=(X_val, Y_val))

In [ ]:
# You can add more layers, DropOut and others

In [ ]:
model.save_weights('model_baseline.h5')

In [ ]:
_, acc = model.evaluate(X_test, Y_test, verbose=0)
print("accuracy on the test set ->", acc * 100.0)

In [ ]:
np.round(model.predict(X_test))

In [ ]:
model.summary()

In [ ]:
# DataPos.dump('DataPos.pkl')
# DataNeg.dump('DataNeg.pkl')

# Test 47 good and 47 bad images.

In [2]:
# get path names for good and bad sources, where good sources are the 
# known sources and the bad sources are the unknown sources

pathToKnownJacobs = 'KnownLenses/Jacobs_KnownLenses/'
pathToKnownDES2017 = 'KnownLenses/DES2017/'
pathToUnknown = 'KnownLenses/Unknown_Processed_256/'

In [3]:
# Loading known examples from Jacobs paper

# r=root, d=directories, f = files

foldersKnownJacobs = []
for root, dirs, files in os.walk(pathToKnownJacobs):
    for folder in dirs:
        foldersKnownJacobs.append(os.path.join(root, folder))

In [4]:
nDT = len(foldersKnownJacobs)
DataKnownJacobs = np.zeros([nDT,3,100,100])

for var in range(len(foldersKnownJacobs)):

    # g_name = get_pkg_data_filename(foldersKnownJacobs[var]+'/g_WCSClipped.fits')
    # r_name = get_pkg_data_filename(foldersKnownJacobs[var]+'/r_WCSClipped.fits')
    # i_name = get_pkg_data_filename(foldersKnownJacobs[var]+'/i_WCSClipped.fits')    

    g_name = get_pkg_data_filename(foldersKnownJacobs[var]+'/g_norm.fits')
    r_name = get_pkg_data_filename(foldersKnownJacobs[var]+'/r_norm.fits')
    i_name = get_pkg_data_filename(foldersKnownJacobs[var]+'/i_norm.fits')    
  
    g = fits.open(g_name)[0].data[0:100,0:100]
    r = fits.open(r_name)[0].data[0:100,0:100]
    i = fits.open(i_name)[0].data[0:100,0:100]    
    
    DataKnownJacobs[var] = [g, r, i]

In [5]:
# Loading DES2017
foldersKnownDES2017 = []
for root, dirs, files in os.walk(pathToKnownDES2017):
    for folder in dirs:
        foldersKnownDES2017.append(os.path.join(root, folder))

In [6]:
nDT = len(foldersKnownDES2017)
DataKnownDES = np.zeros([nDT,3,100,100])

for var in range(len(foldersKnownDES2017)):

    # g_name = get_pkg_data_filename(foldersKnownDES2017[var]+'/g_WCSClipped.fits')
    # r_name = get_pkg_data_filename(foldersKnownDES2017[var]+'/r_WCSClipped.fits')
    # i_name = get_pkg_data_filename(foldersKnownDES2017[var]+'/i_WCSClipped.fits')    

    g_name = get_pkg_data_filename(foldersKnownDES2017[var]+'/g_norm.fits')
    r_name = get_pkg_data_filename(foldersKnownDES2017[var]+'/r_norm.fits')
    i_name = get_pkg_data_filename(foldersKnownDES2017[var]+'/i_norm.fits')    
  
    g = fits.open(g_name)[0].data[0:100,0:100]
    r = fits.open(r_name)[0].data[0:100,0:100]
    i = fits.open(i_name)[0].data[0:100,0:100]    
    
    DataKnownDES[var] = [g, r, i]

In [7]:
DataKnown = np.vstack((DataKnownJacobs, DataKnownDES))

In [8]:
# Loading unknown examples from DES

foldersUnknown = []
for root, dirs, files in os.walk(pathToUnknown):
    for folder in dirs:
        foldersUnknown.append(os.path.join(root, folder))

In [12]:
nDT = len(foldersUnknown)
DataUnknown = np.zeros([nDT,3,100,100])

for var in range(len(foldersUnknown)):
#     g_name = get_pkg_data_filename(foldersUnknown[var]+'/g_WCSClipped.fits')
#     r_name = get_pkg_data_filename(foldersUnknown[var]+'/r_WCSClipped.fits')
#     i_name = get_pkg_data_filename(foldersUnknown[var]+'/i_WCSClipped.fits')    

    g_name = get_pkg_data_filename(foldersUnknown[var]+'/g_norm.fits')
    r_name = get_pkg_data_filename(foldersUnknown[var]+'/r_norm.fits')
    i_name = get_pkg_data_filename(foldersUnknown[var]+'/i_norm.fits')    

    g = fits.open(g_name)[0].data[0:100,0:100]
    r = fits.open(r_name)[0].data[0:100,0:100]
    i = fits.open(i_name)[0].data[0:100,0:100]    
    
    DataUnknown[var] = [g, r, i]

In [15]:
print("DataKnown Mean: " + str(DataKnown.mean()))
print("DataKnown Std: " + str(DataKnown.std()))
print("DataKnown Shape: " + str(DataKnown.shape))

DataKnown Mean: 0.00011520733270134547
DataKnown Std: 1.0001253149096792
DataKnown Shape: (131, 3, 100, 100)


In [16]:
print("DataUnknown Mean: " + str(DataUnknown.mean()))
print("DataUnknown Std: " + str(DataUnknown.std()))
print("DataUnknown Shape: " + str(DataUnknown.shape))

DataUnknown Mean: 0.00020670266772820144
DataUnknown Std: 1.0005700721301807
DataUnknown Shape: (256, 3, 100, 100)


In [ ]:
DataKnown = DataKnown.transpose(0, 3, 2, 1)

y = np.round(model.predict(DataKnown))
Ones = np.count_nonzero(y == 1.)
print("Ones: " + str(Ones))
Zeroes = (np.count_nonzero(y == 0))
print("Zeroes: " + str(Zeroes))


In [ ]:
DataUnknown = DataUnknown.transpose(0, 3, 2, 1 )

y = np.round(model.predict(DataUnknown))
Ones = np.count_nonzero(y == 1.)
print("Ones: " + str(Ones))
Zeroes = (np.count_nonzero(y == 0))
print("Zeroes: " + str(Zeroes))

